In [ ]:
! nvidia-smi

Sun Aug 23 07:25:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Downgrade Tensorflow, because the face align script was written in TF1.x

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
! pip install scipy==1.1.0 numpy==1.16.2

     |████████████████████████████████| 31.2MB 101kB/s 
     |████████████████████████████████| 17.3MB 205kB/s 
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


Get the facenet helper functions, that we will use to align our faces

In [ ]:
! git clone https://github.com/davidsandberg/facenet/ && cd facenet && git checkout 096ed770f163957c1e56efa7feeb194773920f6e

Cloning into 'facenet'...
remote: Enumerating objects: 3149, done.
remote: Total 3149 (delta 0), reused 0 (delta 0), pack-reused 3149
Receiving objects: 100% (3149/3149), 2.94 MiB | 2.00 MiB/s, done.
Resolving deltas: 100% (2230/2230), done.
Note: checking out '096ed770f163957c1e56efa7feeb194773920f6e'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 096ed77 Merge pull request #707 from AntonOellerer/patch-3


## Process My Indian Faces Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! unzip -q '/content/drive/My Drive/eva-4/Phase-2/Session-4/models/train.zip'

count the number of faces

In [ ]:
! ls /content/train | wc -l

10


This script will process and align the faces into `160x160` also writes a bbox file for reference

In [ ]:
%%writefile preprocess.sh
#!/bin/bash
export PYTHONPATH="${PYTHONPATH}:/content/facenet/src"
mkdir -p /content/new_set_data
for N in {1..4}
do
    START=$(date +%s)
    echo "=> pass $N started $(date)"
    python /content/facenet/src/align/align_dataset_mtcnn.py /content/train /content/new_set_data --image_size 160 --margin 32 --random_order --gpu_memory_fraction 0.25 > /dev/null 2>&1
    END=$(date +%s)
    DIFF=$(( $END - $START ))
    echo "=> pass $N completed in $DIFF seconds"
done

Writing preprocess.sh


In [ ]:
! bash preprocess.sh

=> pass 1 started Sun Aug 23 07:31:46 UTC 2020
=> pass 1 completed in 147 seconds
=> pass 2 started Sun Aug 23 07:34:13 UTC 2020
=> pass 2 completed in 5 seconds
=> pass 3 started Sun Aug 23 07:34:18 UTC 2020
=> pass 3 completed in 5 seconds
=> pass 4 started Sun Aug 23 07:34:23 UTC 2020
=> pass 4 completed in 5 seconds


In [ ]:
! zip -rq new_data_final.zip /content/new_set_data 

This script will process the images, compress them to `jpg` and also split them into `train` and `val` folders for training

In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np

target_folder = Path('/content/new_data_final_split')
target_folder.mkdir(parents=True, exist_ok=True)

for pfolder in [fol for fol in Path('/content/lfw_funneled_160').iterdir() if fol.is_dir()]:
    images = list(pfolder.glob('**/*'))
    num_images = len(images)
    if num_images == 1:
        # augment the image since we only have one image
        try:
            img = Image.open(images[0])
            img.convert('RGB')
            out = img.transpose(Image.FLIP_LEFT_RIGHT)

            # save train image
            tr_target_dir = target_folder / 'train' / pfolder.stem
            tr_target_dir.mkdir(parents=True, exist_ok=True)
            img.save(tr_target_dir / f'{pfolder.stem}_001.jpg')

            # save test image
            val_target_dir = target_folder / 'val' / pfolder.stem
            val_target_dir.mkdir(parents=True, exist_ok=True)
            out.save(val_target_dir / f'{pfolder.stem}_002.jpg')

        except:
            pass
    else:
        train_split = int(np.floor(0.7 * num_images))
        train_images = images[:train_split]
        val_images = images[train_split:]

        idx = 0

        # save training images
        for tr_img in train_images:
            try:
                img = Image.open(tr_img)
                img.convert('RGB')
                tr_target_dir = target_folder / 'train' / pfolder.stem
                tr_target_dir.mkdir(parents=True, exist_ok=True)
                img.save(tr_target_dir / f'{pfolder.stem}_{idx:03}.jpg')
                idx += 1
            except:
                pass
        # save validation images
        for val_img in val_images:
            try:
                img = Image.open(val_img)
                img.convert('RGB')
                val_target_dir = target_folder / 'val' / pfolder.stem
                val_target_dir.mkdir(parents=True, exist_ok=True)
                img.save(val_target_dir / f'{pfolder.stem}_{idx:03}.jpg')
                idx += 1
            except:
                pass
print('=> processed files')

=> processed files


In [ ]:
! zip -rq new_data_final_split_lfw.zip /content/new_data_final_split/

In [ ]:
!cp /content/new_data_final_split_lfw.zip '/content/drive/My Drive/eva-4/Phase-2/Session-4'

In [ ]:
!ls -la /content/new_data_final_split/train/alluarjun | wc -l

16


---

<h3><strong>------ DANGER DANGER DANGER DANGER DANGER DANGER ------</strong></h3>

In [ ]:
! rm -r indian_face_dataset

In [ ]:
! rm -r indian_face_split_160

In [ ]:
! rm indian-face-split-160.zip

## Process LFW Dataset

In [ ]:
! wget http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz

--2020-08-23 07:35:09--  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243346528 (232M) [application/x-gzip]
Saving to: ‘lfw-funneled.tgz’

lfw-funneled.tgz    100%[===================>] 232.07M  12.4MB/s    in 24s     

2020-08-23 07:35:34 (9.69 MB/s) - ‘lfw-funneled.tgz’ saved [243346528/243346528]



In [ ]:
! tar -zxf lfw-funneled.tgz

In [ ]:
! ls lfw_funneled | wc -l

5760


In [ ]:
%%writefile preprocess.sh
#!/bin/bash
export PYTHONPATH="${PYTHONPATH}:/content/facenet/src"
mkdir -p /content/lfw_funneled_160
for N in {1..4}
do
    START=$(date +%s)
    echo "=> pass $N started $(date)"
    python /content/facenet/src/align/align_dataset_mtcnn.py /content/lfw_funneled /content/lfw_funneled_160 --image_size 160 --margin 32 --random_order --gpu_memory_fraction 0.25 > /dev/null 2>&1
    END=$(date +%s)
    DIFF=$(( $END - $START ))
    echo "=> pass $N completed in $DIFF seconds"
done

Overwriting preprocess.sh


In [ ]:
! bash preprocess.sh

=> pass 1 started Sun Aug 23 07:35:48 UTC 2020
=> pass 1 completed in 594 seconds
=> pass 2 started Sun Aug 23 07:45:42 UTC 2020
=> pass 2 completed in 5 seconds
=> pass 3 started Sun Aug 23 07:45:47 UTC 2020
=> pass 3 completed in 5 seconds
=> pass 4 started Sun Aug 23 07:45:52 UTC 2020
=> pass 4 completed in 5 seconds


In [ ]:
! tar -cjf lfw-funneled-160.tar.bz2 lfw_funneled_160

---

**DANGER**

In [ ]:
!ls -la /content/lfw_funneled_160 | wc -l

5758


In [ ]:
! rm -r lfw_funneled_160

In [ ]:
! export PYTHONPATH="${PYTHONPATH}:/content/facenet/src" && python /content/facenet/src/align/align_dataset_mtcnn.py /content/lfw_funneled /content/lfw_funneled_160 --image_size 160 --margin 32 --random_order --gpu_memory_fraction 0.25

Streaming output truncated to the last 5000 lines.
/content/lfw_funneled/Sharon_Davis/Sharon_Davis_0001.jpg
/content/lfw_funneled/Sharon_Davis/Sharon_Davis_0002.jpg
/content/lfw_funneled/Jim_Beattie/Jim_Beattie_0001.jpg
/content/lfw_funneled/Dorothy_Wilson/Dorothy_Wilson_0001.jpg
/content/lfw_funneled/Bob_Petrino/Bob_Petrino_0001.jpg
/content/lfw_funneled/Mehmet_Okur/Mehmet_Okur_0001.jpg
/content/lfw_funneled/Bill_Pryor/Bill_Pryor_0001.jpg
/content/lfw_funneled/Gary_Gero/Gary_Gero_0001.jpg
/content/lfw_funneled/Ajit_Agarkar/Ajit_Agarkar_0001.jpg
/content/lfw_funneled/Alan_Stonecipher/Alan_Stonecipher_0001.jpg
/content/lfw_funneled/Jan_Ullrich/Jan_Ullrich_0002.jpg
/content/lfw_funneled/Jan_Ullrich/Jan_Ullrich_0004.jpg
/content/lfw_funneled/Jan_Ullrich/Jan_Ullrich_0005.jpg
/content/lfw_funneled/Jan_Ullrich/Jan_Ullrich_0003.jpg
/content/lfw_funneled/Jan_Ullrich/Jan_Ullrich_0006.jpg
/content/lfw_funneled/Jan_Ullrich/Jan_Ullrich_0001.jpg
/content/lfw_funneled/Bill_Sizemore/Bill_Sizemore_000